In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
from datetime import datetime,timedelta
import time
import concurrent.futures   #多執行序

In [6]:
urls = []
date = datetime.strptime("2017/05/01", "%Y/%m/%d")   #把字串轉成日期
delta = timedelta(days = 1)

while date <= datetime.strptime("2017/11/30", "%Y/%m/%d"):  # time.strftime("%Y/%m/%d")現在日期時間
    resp = requests.get("https://udn.com/news/archive/2/6638/%s/1"%date.strftime("%Y/%m/%d"))  #把時分秒去掉
    resp = BeautifulSoup(resp.text,'html5lib')
    r = resp.select('div.pagelink > span.total')
    s = r[0].text                           
    r = re.findall('共 (.) 頁',s)[0]
    r = int(r)
    for p in range(1,r+1):
        resp = requests.get("https://udn.com/news/archive/2/6638/{}/{}".format(date.strftime("%Y/%m/%d"),p))
        resp = BeautifulSoup(resp.text,'html5lib')
        for url in resp.select('div#ranking_body > table#ranking_table > tbody > tr > td > a'):
                urls.append('https://udn.com' + url.get('href'))
        print("finish {} {}".format(date.strftime("%Y/%m/%d"),p))       

    date += delta

finish 2017/05/01 1
finish 2017/05/01 2
finish 2017/05/01 3
finish 2017/05/02 1
finish 2017/05/02 2
finish 2017/05/02 3
finish 2017/05/03 1
finish 2017/05/03 2
finish 2017/05/03 3
finish 2017/05/04 1
finish 2017/05/04 2
finish 2017/05/04 3
finish 2017/05/05 1
finish 2017/05/05 2
finish 2017/05/05 3
finish 2017/05/06 1
finish 2017/05/06 2
finish 2017/05/07 1
finish 2017/05/07 2
finish 2017/05/08 1
finish 2017/05/08 2
finish 2017/05/08 3
finish 2017/05/09 1
finish 2017/05/09 2
finish 2017/05/09 3
finish 2017/05/10 1
finish 2017/05/10 2
finish 2017/05/10 3
finish 2017/05/10 4
finish 2017/05/11 1
finish 2017/05/11 2
finish 2017/05/11 3
finish 2017/05/12 1
finish 2017/05/12 2
finish 2017/05/12 3
finish 2017/05/13 1
finish 2017/05/13 2
finish 2017/05/14 1
finish 2017/05/15 1
finish 2017/05/15 2
finish 2017/05/16 1
finish 2017/05/16 2
finish 2017/05/16 3
finish 2017/05/17 1
finish 2017/05/17 2
finish 2017/05/17 3
finish 2017/05/18 1
finish 2017/05/18 2
finish 2017/05/18 3
finish 2017/05/19 1


finish 2017/09/30 1
finish 2017/09/30 2
finish 2017/10/01 1
finish 2017/10/01 2
finish 2017/10/02 1
finish 2017/10/02 2
finish 2017/10/02 3
finish 2017/10/03 1
finish 2017/10/03 2
finish 2017/10/04 1
finish 2017/10/04 2
finish 2017/10/05 1
finish 2017/10/05 2
finish 2017/10/06 1
finish 2017/10/06 2
finish 2017/10/06 3
finish 2017/10/07 1
finish 2017/10/07 2
finish 2017/10/08 1
finish 2017/10/08 2
finish 2017/10/09 1
finish 2017/10/09 2
finish 2017/10/10 1
finish 2017/10/10 2
finish 2017/10/10 3
finish 2017/10/11 1
finish 2017/10/11 2
finish 2017/10/11 3
finish 2017/10/12 1
finish 2017/10/12 2
finish 2017/10/12 3
finish 2017/10/13 1
finish 2017/10/13 2
finish 2017/10/14 1
finish 2017/10/14 2
finish 2017/10/15 1
finish 2017/10/15 2
finish 2017/10/16 1
finish 2017/10/16 2
finish 2017/10/17 1
finish 2017/10/17 2
finish 2017/10/17 3
finish 2017/10/18 1
finish 2017/10/18 2
finish 2017/10/18 3
finish 2017/10/19 1
finish 2017/10/19 2
finish 2017/10/19 3
finish 2017/10/20 1
finish 2017/10/20 2


In [7]:
def get_detail_page(url):
    resp = requests.get(url)
    s = BeautifulSoup(resp.text, 'html5lib')
    articles = {}
    articles["id"] = url.split('/')[6]
    articles["title"] = s.select('div#story_body_content > h1.story_art_title')[0].text
    articles["media"] = 'united_daily_news'
    articles["url"] = url
    articles["category"] = "politics"
    
    
    if s.select('div.story_bady_info_author > a'):
        articles["reporter"] = s.select('div.story_bady_info_author > a')[0].text
    else:
        s.select('div.story_bady_info_author > a') == ''
            
    if re.findall('(.+)／(.+)報導',s.select("div.story_bady_info_author")[0].text):
            resp = re.findall('(.+)／(.+)報導',s.select("div.story_bady_info_author")[0].text)
            articles["location"] = resp[0][1]        
    else:
        re.findall('(.+)／(.+)報導',s.select("div.story_bady_info_author")[0].text) == ''
        
        
    articles["datetime"] = s.select('div.story_bady_info_author > span')[0].text.split()[0]
    
    text = ''
    for article in s.findAll('p'):
        text+=article.text
    articles["content"] = text
    
    articles["hash"] = hash(text)
    
    
    datecsv = articles["datetime"].split("-")[0] + articles["datetime"].split("-")[1]
    
    with open('./Downloads/newspaper/United_Newspaper_text_{}.txt'.format(datecsv), 'a',encoding="utf-8")as f:    #將UTF8編碼改成中文
        f.write(json.dumps(articles, ensure_ascii=False))
        f.close()
    
    print("ok")

In [8]:
with concurrent.futures.ThreadPoolExecutor(max_workers=40) as executor:
    executor.map(get_detail_page, urls)

okokok


ok
ok
okokokokokokokok







ok
ok
ok
okokokokokokokok







ok
okokok


ok
okok

okokok


ok
okok
okok

ok
ok

ok
ok
ok
ok
ok
ok
okok

ok
ok
ok
okokokok



okokokokokokokok







ok
ok
okokokokokok





ok
ok
okok

ok
ok
ok
okok
ok

ok
ok
okokokokokok





ok
ok
okok

ok
okokokok



okok
okokok



ok
okokok


okokokok



okok

ok
ok
ok
ok
okok

ok
ok
ok
okokokokok




okokokokokok





ok
okokokok



ok
okok
ok

okokok


ok
ok
ok
ok
ok
okok

okok

okok

ok
ok
ok
okok

okok

okok
ok
ok

ok
ok
okok

ok
okokokokok




ok
okok

ok
okok
ok

okokokokokokokokokokokokokokok














okok

okok

okok

ok
okok

ok
okokokok



okok

ok
okokok


ok
okokokokok




ok
okokok


ok
ok
ok
ok
ok
okok

okokokok



ok
ok
okok

okok

ok
ok
okokok


ok
ok
ok
ok
ok
ok
okokokok


okokokokokokokok







ok

okok

okok

ok
ok
ok
ok
okokok


okokokokokok





ok
ok
okok

okokokokokokok





ok
ok
okok


okokok
okok

ok

okok


ok
okokok


ok
ok
okokokokokokok






ok
ok
okokok


okokok

ok
ok

ok
ok
okok

okok

ok
okokokokokokok






ok
okokok


ok
okok

ok
okokokok



ok
ok
okokok


ok
ok
ok
ok
okok

ok
ok
ok
okokokok



ok
ok
ok
ok
ok
okokok


ok
ok
okok
okokokokok





ok
ok
ok
okokok


ok
ok
ok
ok
ok
ok
okok

okokok


ok
okok

ok
ok
ok
okokok

okok


okokok


okok
ok
ok
okokokok




ok
ok
ok
okok

ok
ok
ok
ok
ok
okok

ok
ok
ok
ok
okok

okokok


ok
ok
okok

okok

ok
ok
ok
ok
ok
ok
okokokokok




okok

okok
ok
ok
ok

okok

ok
okok

ok
ok
okok
ok

ok
ok
ok
ok
ok
ok
ok
okok

ok
okok

ok
ok
ok
ok
ok
ok
okok

okok

ok
ok
ok
ok
ok
ok
ok
okokok


okokokok



ok
ok
okok

ok
okok

okok

ok
ok
okokok

ok

ok
ok
ok
ok
ok
ok
ok
ok
okokok


ok
okok

ok
okok

okokokokok




ok
ok
okokok


okok

ok
ok
okokok

ok

okokok


okok

ok
okokok


ok
ok
ok
ok
ok
ok
okokokokok




ok
okok

okok

ok
okokokokok




ok
ok
okokokok



ok
ok
ok
ok
ok
okok

okok

ok
ok
ok
ok
ok
ok
okok

ok
okokokokokokokok







ok
ok
ok
ok
okokok


okok

okokokokokok





ok
okok

ok
ok
ok
okok

ok
ok
ok
okok

okok

okok

ok
okokok


okok

ok
ok
ok
ok
okok

okokok
okok


ok
okokokokok





ok
okokokok



okok

okok

okok
ok

ok
ok
okokokokokokokok







ok
okokok


okokok


okok

ok
ok
ok
okokokokok




ok
ok
okok

okok
ok

okok

okokok


ok
okokok


ok
ok
okokokok



okokok

ok

ok
ok
ok
ok
ok
ok
ok
ok
ok
okokokokok




okok

ok
okokokok



ok
okokokok



ok
ok
okok

ok
ok
okok

ok
okokok


okokok


ok
ok
okokok

okokok



okok

okok

ok
okok
ok
okok


okokok

ok
okokok



ok
okok

ok
ok
okok
okok
okok
ok
ok



ok
okok

ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

okok
okok

ok
okok

ok

ok
okokok


okokokok



ok
ok
ok
ok
okokokok



ok
okok

okok

ok
ok
okok

ok
okokok


ok
ok
ok
ok
ok
okokok


okokokok



ok
ok
okokokokok




ok
ok
okokok

okokok



ok
ok
ok
okok
ok

ok
okok

ok
ok
okok

ok
ok
okokok


ok
ok
ok
ok
ok
okok

ok
ok
ok
okok

ok
ok
okokokok



okok

ok
ok
okok

ok
ok
ok
okok

okok

ok
okok

ok
ok
ok
ok
ok
ok
okok

okok
ok
ok

ok
okok

okokok


ok
ok
ok
okokok


okokokokokok





ok
okokok


o

okok
ok
okok

ok
ok
ok

okokok

okokok


ok
okok

ok
ok

ok
ok
ok
ok
okokok


ok
okok
ok

ok
okok

ok
ok
okokokokokok





ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

ok
okok

ok
ok
ok
okok

okokokokokokokok







ok
ok
okokok


ok
ok
ok
okok

ok
ok
okokokokok




okokok


ok
okok

ok
ok
ok
ok
ok
okokokokokokok






ok
ok
okokok


okok

ok
ok
okokok


ok
ok
ok
okok

ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

okokok


okok

ok
ok
okokokok



ok
okok

ok
okok

okok

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

ok
ok
okokok


ok
ok
okok

ok
ok
ok
okok

ok
ok
okok

okok

ok
ok
okok

okok

ok
okokokok



ok
ok
ok
ok
ok
okok

ok
ok
ok
okokok


ok
ok
okok

ok
ok
ok
ok
okokok


okok

ok
ok
okokok


ok
okokok


okokok


ok
ok
ok
ok
okok
okok
ok


okokokokokokokokokokok










ok
okokokok



ok
okokok


ok
ok
okokok

okokok



ok
okokok


ok
ok
ok
ok
okokokokok




okok

okok

ok
okok

okok

ok
ok
ok
okokokok



ok
okokokokokok





okokokok



ok
ok
okok

ok
ok
okokok
ok


ok
ok
ok
okokok


ok
okokokok



ok
ok
o

okok

ok
okok

okok
ok

okokokokokokok






ok
okokok

okok


ok
okok

okokok


ok
ok
ok
ok
ok
ok
ok
okok
okokokok
ok




okokok


ok
ok
okokok


okok
okok


ok
okokokokok




ok
okokok


ok
ok
ok
ok
okokok


ok
ok
ok
ok
ok
okokokokok




ok
ok
ok
okokok


okok

okokokokokokokokok








ok
okok

ok
ok
okokokok



ok
ok
ok
okokokokokok





ok
ok
ok
okokokok


ok

ok
ok
ok
ok
ok
okokok


ok
ok
ok
okok

ok
okok

ok
okok

ok
ok
okok

ok
ok
ok
ok
okokokok



ok
okokok


okokokokok




ok
ok
ok
ok
okok

ok
ok
ok
ok
ok
ok
ok
okokokokok




ok
ok
okok

ok
okok

ok
okok
okokokokok




ok

ok
ok
ok
ok
okokok


ok
ok
ok
ok
okok

ok
ok
ok
ok
okok
okokok



ok
ok
ok
ok
okok

ok
ok
ok
ok
ok
okokok

ok
okokokokokokok







okok

ok
okok
ok

okokokok


ok
ok

ok
okok
ok

ok
ok
okok

ok
okokokokok




ok
ok
okokokokok




okokok


ok
okokok


okok
ok

ok
ok
okok

ok
ok
okok

ok
ok
ok
okok

ok
ok
okokokokok
ok




ok
ok
okokokokok



ok
ok
ok

ok
okok

ok
okokokok



ok
ok
ok
ok
ok
ok
ok
okokok


o

...

okokokokok




okok

okokokok



okok

ok
okok

okok

okok

okokok


ok
ok
okok

okok
ok
ok

ok
okok

ok
ok
okokokokokokokok






ok

okok

ok
ok
okok

ok
okok
ok

okokok


ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

ok
ok
ok
okok

okok

okok
ok

okokokok



okok

okokok


ok
okok

okok
ok

ok
okok

okokokokokok





okokok


okok

ok
okok

ok
okokokok



ok
ok
okok

okokokok



ok
ok
ok
ok
okok
ok
ok

ok
okok

okokokokok




ok
okokokokokokok





ok

ok
okokok

ok
ok
ok

ok
okokokokok




ok
okok
ok

okok

okokokok



ok
ok
ok
ok
okokokokokokokokokok








ok

ok
okokokokokokokok






ok

ok
ok
okokok


ok
ok
ok
okokokokokokokok






ok
ok

ok
ok
okokok
ok
ok

ok
okokokok




okok
ok
okok
okok



okokokok


ok

ok
ok
okokok


ok
ok
okokokokokokokok







ok
okok

ok
okokok


ok
ok
okokokokokokokokok








ok
okokok


okok

okokokokok




okok

okok

ok
ok
okok

ok
okok

okok

okokok


ok
ok
ok
okokok


ok
ok
ok
ok
okokok


okokokok



ok
ok
okok

ok
ok
okokok


ok
ok
ok
ok
ok
ok
ok
ok
o

ok
okokokokok



okokokok
ok



ok
ok
ok
ok

okok

okok

ok
okok

ok
okokokokok




ok
ok
okok

okok

ok
okokokok



ok
ok
okok

ok
ok
ok
okok

ok
okok

ok
ok
ok
okokokok



okok

ok
ok
ok
ok
okok

ok
okok

okok

ok
ok
ok
ok
ok
okok
ok
ok
ok
okok


ok
ok
ok
okok

okok
ok

okokokokok




ok
okokok

ok

ok
okok

ok
ok
ok
ok
okok

okok

ok
okok

ok
okokok


okokok


ok
ok
ok
ok
ok
okokokokok




ok
okokokokok



ok
ok
ok
okokok



okok

ok
ok
ok
ok
okokokok



ok
ok
ok
ok
ok
ok
okok
ok

okokokokokok





ok
okok

okokok


ok
ok
ok
okok

okok

ok
ok
okokok

ok

ok
ok
okok

ok
okok
okokok



ok
ok
ok
okokok


ok
ok
ok
okokokok



ok
ok
okokokokokok





ok
okok

ok
okok
ok

ok
okok

ok
okokok
ok
ok

ok
okokok



ok
ok
ok
ok
ok
okok

ok
okokokokokokok






ok
ok
ok
okok

ok
okokok


ok
okokok


okokokokok




ok
ok
ok
okok

ok
ok
okok

ok
ok
ok
ok
ok
ok
okokok


ok
ok
okokok


ok
okok

ok
ok
okok

okok

ok
okok
ok
okok
ok

okok


ok
okokok


ok
ok
okok

okokokokok




okokok


okokokokokoko

ok
ok
okok

ok
ok
okokokokok




okok

okokok


okokokokokokokok







ok
ok
ok
ok
ok
okokok


ok
okok
ok

ok
ok
okok

ok
okokokok



ok
okok

ok
okokokokok




ok
ok
okok

okok

ok
okok

okokokokok




okokokok



ok
ok
ok
okokokok
ok
ok


ok

ok
ok
ok
ok
okokok


okokokokokok





ok
ok
ok
okok

ok
okokok


ok
okok

ok
okok

ok
okok

okok

ok
okok

ok
ok
okok

ok
okokokok



ok
okokokok



ok
ok
ok
ok
ok
okok

ok
okok
ok
ok

okokok


okok

ok
okok

ok
okokok


ok
ok
okokok


ok
okok

ok
okokokok


ok
ok

ok
ok
ok
ok
ok
okokok


ok
okokokok


okokok



okokokok



ok
ok
ok
okok

ok
okokok


okokokokokokokokok








ok
ok
okokokok



ok
okok

ok
okok

ok
okok

okok

ok
ok
ok
okokok

ok
ok

okokok


ok
okokok


ok
okok

ok
ok
okokok


ok
ok
okokok


okokok


ok
okokokok



okok

okok

ok
ok
ok
ok
okokokokok




ok
okok

ok
okokokokok




okok

okokok


ok
okok

ok
ok
ok
ok
ok
okok

ok
ok
okokokok



okok

ok
okokok

okokok



ok
okokokokok




ok
ok
okok

okokokokokokok






okokoko

okokok


ok
okokok


ok
okokok


ok
ok
ok
ok
okokokok



okokokok



ok
okok

ok
okok

okokok


okokokok



okok

ok
ok
ok
okokokokok




okokokokokok





ok
ok
okok

ok
okok

ok
okok

ok
ok
ok
okokok


ok
okok

ok
okokokok


okok


ok
okokok


ok
ok
ok
ok
ok
okok

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

ok
okokokok



ok
ok
ok
ok
ok
okok

ok
ok
ok
okok

ok
ok
okok

okok

okokokok



okok

okok

okok

ok
ok
ok
ok
ok
ok
okok

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

okok

ok
okokok


ok
ok
ok
ok
ok
ok
ok
okok

okokokok



ok
okok
ok

ok
okok
okok

ok

ok
okok

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
okok

ok
okok

okok

okok

ok
ok
okokok


ok
ok
ok
ok
okok

okokok


ok
ok
okok

ok
okok

okokok


ok
okokokok



okokok


ok
okokokokokokok






ok
ok
okok

ok
ok
ok
ok
ok
ok
ok
okok
ok
ok
okok


ok
okokok


okokokok



okok

ok
okok

ok
o

ok
okok

okokok

ok
okokokokokokokokok








ok

ok
okokokokok




okok

ok
ok
okokokokokokokokokok









okok

okok

ok
ok
okok
okokok



ok
ok
okok

ok
ok
okok
ok

ok
ok
ok
ok
ok
ok
okokokokokok





ok
okokokok



okokok


ok
okok

ok
ok
okokok


ok
okokok

okok


ok
ok
okok

okok

ok
okokokokok




ok
ok
ok
okokokok



ok
okok

okokokokokokokok







okok

ok
ok
okok

okok
ok

ok
ok
ok
okok

ok
okokokok



ok
okokok


okokokokokokok






okok

ok
okokokokok




okok

ok
okok
ok

ok
ok
okok

okokok

okok


ok
ok
okokokokokok





ok
ok
ok
okok

ok
ok
okok

ok
ok
ok
okok

okok

okokokokokok





ok
ok
ok
ok
okokokokokokokok







okokokokokokokokok



okok


okokokok






okokokokok





okokokok



okok

ok
okokok


ok
ok
okok
ok
ok

ok
ok
okok

ok
ok
ok
ok
okokokokokok




ok

okokokokok




ok
okok
okok


ok
ok
ok
ok
okokokok



ok
ok
okok

ok
okokok

okok
okokok




okokokokokokokokokokok










ok
ok
okok
ok

ok
ok
ok
ok
okokokokokokokok







ok
okokokok


ok

okok